In [21]:
# prompt template
# langchain 에서 유효성 검사도 해준다. 
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()
# method1 : 기본적인 방법으로 prompt template 을 생성합니다.
t = PromptTemplate(template="Hello, what do you know about this {country}?",input_variables=["country"])


# method2 : from_template 을 사용하여 prompt template 을 생성합니다.
t2 = PromptTemplate.from_template("Hello, what do you know about this {country}?")

message = t2.format(country="Kongo")
chat.predict(message)

"The Kongo, also known as the Kingdom of Kongo, was a pre-colonial kingdom located in west central Africa. It was founded in the 14th century and reached its peak in the 16th century under the rule of King Afonso I. The kingdom was known for its centralized government, strong military, and trade networks with European powers.\n\nThe Kongo Kingdom was Christianized by Portuguese missionaries in the 16th century, making it one of the first African kingdoms to convert to Christianity. The kingdom had a complex social structure with a king at the top, followed by nobles, commoners, and slaves.\n\nThe kingdom eventually declined due to internal power struggles, civil wars, and the impact of the transatlantic slave trade. It was formally annexed by Portugal in the late 19th century and became part of the Portuguese colony of Angola.\n\nToday, the descendants of the Kongo people can be found in modern-day Angola, the Democratic Republic of Congo, and the Republic of Congo. The kingdom's legac

In [ ]:
#✅few shot prompt template 
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
  { "question":"what do you know about France?",
    "answer":"""
    France is a country in Europe.
    food : baguette, cheese, wine
    capital : Paris
    language : French
    currency : Euro
    """
  },
  { "question":"what do you know about Korea?",
    "answer":"""
    Korea is a country in Asia.
    food : Kimchi, Bibimbap
    capital : Seoul
    language : Korean
    currency : Won
    """
  },
  { "question":"what do you know about Japan?",
    "answer":"""
    Japan is a country in Asia.
    food : Sushi, Ramen
    capital : Tokyo
    language : Japanese
    currency : Yen
    """
  }
]

example_template ="""
Human : {question}
AI : {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
# example 들을 example prompt 형식으로 변환해줍니다.
prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples, 
  suffix="what do you know about {country}",
  input_variables=["country"])

prompt.format(country="switzerland")
chain = prompt | chat
chain.invoke({"country":"Ghana"})



AIMessage(content='AI: \n    Ghana is a country in West Africa.\n    food: Fufu, Jollof rice\n    capital: Accra\n    language: English\n    currency: Ghanaian Cedi')

In [27]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


prompt = ChatPromptTemplate.from_messages([("human" ,"What do you know about {country}"), ("ai" , "{answer}")])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly and detailed geography expert. After listing basic facts, you always add an interesting explanation or cultural insight about the country."), 
    example_prompt,
    ("human","What do you know about {country}?"),])

chain = final_prompt | chat

chain.invoke({"country": "Turkey"})


        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

Turkey is a country that straddles both Europe and Asia, giving it a unique cultural blend. It has a rich history, with landmarks such as the Hagia Sophia and the ancient city of Ephesus. Turkish cuisine is known for its delicious flavors and diverse dishes, influenced by Ottoman, Middle Eastern, and Mediterranean cuisines. Kebabs, baklava, and Turkish delight are just a few of the popular dishes you can enjoy in Turkey.

AIMessageChunk(content='\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira\n\nTurkey is a country that straddles both Europe and Asia, giving it a unique cultural blend. It has a rich history, with landmarks such as the Hagia Sophia and the ancient city of Ephesus. Turkish cuisine is known for its delicious flavors and diverse dishes, influenced by Ottoman, Middle Eastern, and Mediterranean cuisines. Kebabs, baklava, and Turkish delight are just a few of the popular dishes you can enjoy in Turkey.')

In [ ]:
# ✅dynamic example selector
# example이 길어질 수록 비용이 많이 들고, 모델에 허용되는 범위가 있기 때문 

from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 예제를 특정 길이 조건에 맞는 것만 선택합니다.
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt =example_prompt, 
    max_length=300
)


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")


prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Greece?\nAI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [ ]:

from langchain.prompts import example_selector
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

In [41]:
#✅ 외부에서 prompt 불러오기 
from langchain.prompts import load_prompt
# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

prompt.format(country="Cocoa Beach")

'What is the capital of Cocoa Beach'

In [57]:
#✅prompt 파이프라인 구축하기
from langchain.prompts import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.6,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    당신은 성경 인물 {character}의 말투와 성품을 따라 대답하는 AI입니다.
    말씀과 삶을 바탕으로 따뜻하고 지혜롭게 답변해주세요.
    줄바꿈을 잘 넣어주세요
    """
)

example = PromptTemplate.from_template(
    """
    예시:

    질문: 억울한 일을 당했을 땐 어떻게 해야 하나요?
    답변: (요셉처럼) 저도 누명을 쓰고 감옥에 갇힌 적이 있습니다.
    하지만 하나님께서 모든 상황을 선하게 바꾸실 것을 믿었기에 참고 기다릴 수 있었습니다.
    형통함은 결국 하나님께서 주시는 것이니까요.
    """
)

start = PromptTemplate.from_template(
    """
    지금부터 질문에 대답해주세요.

    질문: {question}
    답변:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "다윗",
        "example_question": "너 2025년에 다시 태어나면 어떻게 살거같아?",
        "example_answer": """
        2025년에도 하나님을 찬양하고 
        하나님의 뜻을 따르는 것이 중요하다고 생각해.
        """,
        "question": "다윗 나에게 성경구절을 인용해서 오늘 조언 한마디 해줘",
    }
)

(잠언 3:5-6) "온 마음으로 여호와를 의뢰하라. 네가 이해하는 바 모든 일에 의지하지 말라. 그의 도리로 인하여 행하라. 그는 네 길을 지시하시리라." 
하나님을 의뢰하고 그의 도리를 따르는 것이 가장 중요합니다. 언제나 하나님을 바라보며 의지하며 그의 뜻대로 행하면 모든 일이 잘 풀리게 될 것입니다.

AIMessageChunk(content='(잠언 3:5-6) "온 마음으로 여호와를 의뢰하라. 네가 이해하는 바 모든 일에 의지하지 말라. 그의 도리로 인하여 행하라. 그는 네 길을 지시하시리라." \n하나님을 의뢰하고 그의 도리를 따르는 것이 가장 중요합니다. 언제나 하나님을 바라보며 의지하며 그의 뜻대로 행하면 모든 일이 잘 풀리게 될 것입니다.')

In [ ]:
# ✅ 비용 절감을 위해 캐싱하기
